# Tools
- ToolConfig, ToolResponse - in converse/converse_stream
- toolChoice - can be used to enforce specific tool, or offer suite of tools to LLM

In [1]:
import os
import boto3
from dotenv import load_dotenv

# Client Setup
load_dotenv()

client = boto3.client("bedrock-runtime", region_name="us-east-1",
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
                      )

inference_profile_id = 'us.anthropic.claude-3-5-haiku-20241022-v1:0' # inference profile
model_id = inference_profile_id

In [2]:
# Helper functions

def add_user_message(messages, content):
    if isinstance(content, str):
        user_message = {"role": "user", "content": [{"text": content}]}
    else:
        user_message = {"role": "user", "content": content}
    messages.append(user_message)


def add_assistant_message(messages, content):
    if isinstance(content, str):
        assistant_message = {
            "role": "assistant",
            "content": [{"text": content}],
        }
    else:
        assistant_message = {"role": "assistant", "content": content}

    messages.append(assistant_message)


def chat(messages, system=None, temperature=1.0, stop_sequences=[], tools=None):
    params = {
        "modelId": model_id,
        "messages": messages,
        "inferenceConfig": {
            "temperature": temperature,
            "stopSequences": stop_sequences,
        },
    }

    if system:
        params["system"] = [{"text": system}]

    if tools:
        params["toolConfig"] = {"tools": tools, "toolChoice": {"auto": {}}}

    response = client.converse(**params)
    parts = response["output"]["message"]["content"]

    return {
        "parts": parts,
        "stop_reason": response["stopReason"],
        "text": "\n".join([p["text"] for p in parts if "text" in p]),
    }

In [ ]:
# Tools and Schemas

from datetime import datetime, timedelta


def add_duration_to_datetime(
    datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
    date = datetime.strptime(datetime_str, input_format)

    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29
                if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
                else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    print(
        f"----\nSetting the following reminder for {timestamp}:\n{content}\n----"
    )


get_current_datetime_schema = {
    "toolSpec": {
        "name": "get_current_datetime",
        "description": "Retrieves the current date and time formatted according to the specified format string. This tool returns the system's current local date and time as a formatted string. It should be used when the user needs to know the current date, time, or both in a specific format. The tool uses Python's datetime.now() function internally, so it will reflect the server's local time zone settings. This tool does not support retrieving dates or times for different time zones or historical/future dates.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "date_format": {
                        "type": "string",
                        "description": "A string that specifies the format of the returned date and time, using Python's strftime format codes. For example, '%Y-%m-%d' returns only the date as '2025-04-02', '%H:%M:%S' returns only the time as '14:30:45', and the default '%Y-%m-%d %H:%M:%S' returns both as '2025-04-02 14:30:45'. Use format codes like %Y (4-digit year), %m (month), %d (day), %H (hour), %M (minute), and %S (second) to customize the output format.",
                    }
                },
            },
        },
    }
}


add_duration_to_datetime_schema = {
    "toolSpec": {
        "name": "add_duration_to_datetime",
        "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "datetime_str": {
                        "type": "string",
                        "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
                    },
                    "duration": {
                        "type": "number",
                        "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
                    },
                    "unit": {
                        "type": "string",
                        "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
                    },
                    "input_format": {
                        "type": "string",
                        "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
                    },
                },
                "required": ["datetime_str"],
            }
        },
    }
}

set_reminder_schema = {
    "toolSpec": {
        "name": "set_reminder",
        "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
                    },
                    "timestamp": {
                        "type": "string",
                        "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
                    },
                },
                "required": ["content", "timestamp"],
            }
        },
    }
}

batch_tool_schema = {
    "toolSpec": {
        "name": "batch_tool",
        "description": "Invoke multiple other tool calls simultaneously",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "invocations": {
                        "type": "array",
                        "description": "The tool calls to invoke",
                        "items": {
                            "types": "object",
                            "properties": {
                                "name": {
                                    "types": "string",
                                    "description": "The name of the tool to invoke",
                                },
                                "arguments": {
                                    "types": "string",
                                    "description": "The arguments to the tool, encoded as a JSON string",
                                },
                            },
                            "required": ["name", "arguments"],
                        },
                    }
                },
                "required": ["invocations"],
            }
        },
    }
}

batch_tool_schema = {
    "toolSpec": {
        "name": "batch_tool",
        "description": "Invoke multiple other tool calls simultaneously",
        "inputSchema": {
            "json": {
                "type": "object",
                "properties": {
                    "invocations": {
                        "type": "array",
                        "description": "The tool calls to invoke",
                        "items": {
                            "types": "object",
                            "properties": {
                                "name": {
                                    "types": "string",
                                    "description": "The name of the tool to invoke",
                                },
                                "arguments": {
                                    "types": "string",
                                    "description": "The arguments to the tool, encoded as a JSON string",
                                },
                            },
                            "required": ["name", "arguments"],
                        },
                    }
                },
                "required": ["invocations"],
            }
        },
    }
}

In [ ]:
def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    return datetime.now().strftime(date_format)


get_current_datetime()

In [5]:
import json


def run_batch(tool_input):
    batch_output = []
    for invocation in tool_input["invocations"]:
        tool_name = invocation["name"]
        args = json.loads(invocation["arguments"])

        tool_output = run_tool(tool_name, args)
        batch_output.append({"tool_name": tool_name, "output": tool_output})

    return batch_output


def run_tool(tool_name, tool_input):
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    elif tool_name == "set_reminder":
        return set_reminder(**tool_input)
    elif tool_name == "add_duration_to_datetime":
        return add_duration_to_datetime(**tool_input)
    elif tool_name == "batch_tool":
        return run_batch(tool_input)
    else:
        raise Exception(f"Unknown tool name: {tool_name}")


def run_tools(parts):
    tool_requests = [part for part in parts if "toolUse" in part]
    tool_result_parts = []

    for tool_request in tool_requests:
        tool_use_id = tool_request["toolUse"]["toolUseId"]
        tool_name = tool_request["toolUse"]["name"]
        tool_input = tool_request["toolUse"]["input"]

        try:
            tool_output = run_tool(tool_name, tool_input)
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    "content": [{"text": json.dumps(tool_output)}],
                    "status": "success",
                }
            }
        except Exception as e:
            tool_result_part = {
                "toolResult": {
                    "toolUseId": tool_use_id,
                    "content": [{"text": f"Error: {e}"}],
                    "status": "error",
                }
            }

        tool_result_parts.append(tool_result_part)

    return tool_result_parts


In [ ]:
def run_conversation(messages):
    while True:
        result = chat(
            messages,
            tools=[
                get_current_datetime_schema,
                add_duration_to_datetime_schema,
                set_reminder_schema,
                batch_tool_schema,
            ],
        )

        add_assistant_message(messages, result["parts"])
        print(result["text"])

        if result["stop_reason"] != "tool_use":
            break

        tool_result_parts = run_tools(result["parts"])
        add_user_message(messages, tool_result_parts)

    return messages

# Multi-Turn Conversation with tools

In [ ]:
messages = []

add_user_message(
    messages,
    "What is today's date + 50 days? Also, what is March 12th, 2025 + 100 days?",
)


run_conversation(messages)

[{'text': "I'll help you set a reminder for your appointment. To do this correctly, I need to calculate the date that is a week from Tuesday.\n\nLet me first determine what date the next Tuesday is, and then add a week to that."}, {'toolUse': {'toolUseId': 'tooluse_e7BSmcBCTWCKas7Jbwecjw', 'name': 'get_current_datetime', 'input': {'date_format': '%Y-%m-%d %A'}}}]
[{'text': "Now I'll calculate when the next Tuesday is:\n\nTuesday will be on 2025-04-22, as it's the next Tuesday after today (Friday).\n\nA week from that Tuesday would be:"}, {'toolUse': {'toolUseId': 'tooluse_B1nyIsZQS16e85dRALrFEA', 'name': 'add_duration_to_datetime', 'input': {'datetime_str': '2025-04-22', 'duration': 7, 'input_format': '%Y-%m-%d', 'unit': 'days'}}}]
[{'text': "Now I'll set the reminder for your appointment:"}, {'toolUse': {'toolUseId': 'tooluse_x_IBJtaPSASZPJHPosQ5yA', 'name': 'set_reminder', 'input': {'content': 'Appointment', 'timestamp': '2025-04-29T12:00:00'}}}]
----
Setting the following reminder f

[{'role': 'user',
  'content': [{'text': 'Remind me about my appointment. It is a week from Tuesday'}]},
 {'role': 'assistant',
  'content': [{'text': "I'll help you set a reminder for your appointment. To do this correctly, I need to calculate the date that is a week from Tuesday.\n\nLet me first determine what date the next Tuesday is, and then add a week to that."},
   {'toolUse': {'toolUseId': 'tooluse_e7BSmcBCTWCKas7Jbwecjw',
     'name': 'get_current_datetime',
     'input': {'date_format': '%Y-%m-%d %A'}}}]},
 {'role': 'user',
  'content': [{'toolResult': {'toolUseId': 'tooluse_e7BSmcBCTWCKas7Jbwecjw',
     'content': [{'text': '"2025-04-18 Friday"'}],
     'status': 'success'}}]},
 {'role': 'assistant',
  'content': [{'text': "Now I'll calculate when the next Tuesday is:\n\nTuesday will be on 2025-04-22, as it's the next Tuesday after today (Friday).\n\nA week from that Tuesday would be:"},
   {'toolUse': {'toolUseId': 'tooluse_B1nyIsZQS16e85dRALrFEA',
     'name': 'add_duratio